#  Submit geo opt for molecules, slabs and bulks

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# AiiDA imports.
%aiida

# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDAlab imports.
import aiidalab_widgets_base as awb

# Custom imports.
from widgets.build_slab import BuildSlab
from widgets.computational_resources import ProcessResourcesWidget, ResourcesEstimatorWidget
from widgets.inputs import InputDetails
from widgets.empa_viewer import EmpaStructureViewer
from widgets.ANALYZE_structure import StructureAnalyzer
from widgets.import_cdxml import CdxmlUpload2GnrWidget

In [ ]:
Cp2kMoleculeOptWorkChain = WorkflowFactory('nanotech_empa.cp2k.molecule_opt')
Cp2kSlabOptWorkChain = WorkflowFactory('nanotech_empa.cp2k.slab_opt')
Cp2kBulkOptWorkChain = WorkflowFactory('nanotech_empa.cp2k.bulk_opt')

In [ ]:
# Structure selector.
empa_viewer = EmpaStructureViewer()
build_slab = BuildSlab(title='Build slab')
ipw.dlink((empa_viewer, 'details'), (build_slab, 'details'))
ipw.dlink((empa_viewer, 'structure'), (build_slab, 'molecule'))

structure_selector = awb.StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.OptimadeQueryWidget(embedded=True),
        awb.SmilesWidget(title="From SMILES"),
        CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors = [
        awb.BasicStructureEditor(title="Edit structure"),
        build_slab
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computational_resources = awb.ComputationalResourcesWidget(input_plugin='cp2k')
resources = ProcessResourcesWidget()

input_details = InputDetails()
create_input = ipw.Button(description="Create Input")

# Resources

In [ ]:
# Resources estimation.
MAX_NODES=96

structure_analyzer = StructureAnalyzer()

def update_resources(_):

    if not (structure_selector.structure and computational_resources.value):
        node_estimate_message.message = """<span style="color:red"> Error:</span> Can't estimate resources: both structure and code must be selected."""
        return

   
    
    structure_analyzer.structure = substructure
    calctype = "slab" 
    resources = cp2k_utils.get_nodes(
        atoms=structure_selector.structure,
        calctype=calctype,
        computer=computational_resources.value.computer,
        max_nodes=MAX_NODES,
        uks=uks_switch.value
    )
    slab.nodes_widget.value = resources[0]
    slab.cpus_per_node_widget.value = resources[1]

estimate_nodes_button = ipw.Button(description="Estimate resources", button_style='warning')
estimate_nodes_button.on_click(update_resources)
node_estimate_message = awb.utils.StatusHTML()

In [ ]:
# Resources widgets.
STYLE = {'description_width': '100px'}
nodes_widget = ipw.IntText(
    description="# Nodes",
    value=1,
    min=1,
    style=STYLE
    )
cpus_per_node_widget = ipw.IntText(
    description="# CPUs per node",
    value=1,
    min=1,
    style=STYLE
    )
num_cores_per_mpiproc_widget = ipw.IntText(
    description="# thredas",
    value=1,
    min=0,
    style=STYLE
)    
run_time_widget = ipw.IntText(
    description="Runtime (mins)",
    value=1440,
    min=0,
    style=STYLE
)

In [ ]:
res=ipw.VBox([ipw.HTML("<b>System</b>"),
                nodes_widget,
                cpus_per_node_widget,
                num_cores_per_mpiproc_widget,
                run_time_widget])

In [ ]:
ipw.dlink((empa_viewer, 'details'), (input_details, 'details'))
ipw.dlink((computational_resources, 'value'),(input_details, 'selected_code'))

def prepare_mol_opt():

    builder = Cp2kMoleculeOptWorkChain.get_builder()
    parameters = input_details.final_dictionary.copy()
    builder.code = computational_resources.value
    builder.structure = structure_selector.structure_node
    builder.charge = Int(parameters['charge'])
    
    ase_geom =  structure_selector.structure        

    # spin guess
    mag_list = [ 0 for t in ase_geom ]
    if parameters['multiplicity']>0:
        for i in awb.utils.string_range_to_list(parameters['spin_u'])[0]:
            mag_list[i] = 1
        for i in awb.utils.string_range_to_list(parameters['spin_d'])[0]:
            mag_list[i] = -1 
        builder.multiplicity = Int(parameters['multiplicity'])
        
    builder.magnetization_per_site = List(list=mag_list)    
    builder.vdw = Bool(parameters['vdw_switch'])
    builder.protocol = Str(parameters['protocol'])
    builder.options = {
            "max_wallclock_seconds": resources.walltime_seconds,
            "resources": {
                'num_machines': resources.nodes,
                'num_mpiprocs_per_machine': resources.tasks_per_node,
                'num_cores_per_mpiproc': resources.threads_per_task,
            },
        }
    builder.metadata.label = 'Cp2kMoleculeOptWorkChain'
    builder.metadata.description = parameters['description']

    return builder

def prepare_slab_opt():

    builder = Cp2kSlabOptWorkChain.get_builder()
    parameters = input_details.final_dictionary.copy()
    builder.code = computational_resources.value
    builder.structure = structure_selector.structure_node
    builder.charge = Int(parameters['charge'])
    
    ase_geom =  structure_selector.structure        

    # spin guess
    mag_list = [ 0 for t in ase_geom ]
    if parameters['multiplicity']>0:
        for i in awb.utils.string_range_to_list(parameters['spin_u'])[0]:
            mag_list[i] = 1
        for i in awb.utils.string_range_to_list(parameters['spin_d'])[0]:
            mag_list[i] = -1 
        builder.multiplicity = Int(parameters['multiplicity'])
        
    builder.magnetization_per_site = List(list=mag_list)    
    builder.vdw = Bool(parameters['vdw_switch'])
    builder.fixed_atoms = Str(parameters['fixed_atoms'])
    builder.protocol = Str(parameters['protocol'])
    builder.options = {
            "max_wallclock_seconds": resources.walltime_seconds,
            "resources": {
                'num_machines': resources.nodes,
                'num_mpiprocs_per_machine': resources.tasks_per_node,
                'num_cores_per_mpiproc': resources.threads_per_task,
            },
        }
    
    builder.metadata.label = 'Cp2kSlabOptWorkChain'
    builder.metadata.description = parameters['description'] 

    return builder

def prepare_bulk_opt():

    builder = Cp2kBulkOptWorkChain.get_builder()
    parameters = input_details.final_dictionary.copy()
    builder.code = computational_resources.value
    builder.structure = structure_selector.structure_node
    
    
    # cell opt
    if parameters['opt_cell']:
        builder.cell_opt = Bool(True)
        builder.symmetry = Str(parameters['cell_sym'])
        builder.cell_freedom = Str(parameters['cell_free'])
    
    # fixed atoms only in bulk opt
    else:
        builder.fixed_atoms = Str(parameters['fixed_atoms'])
    
    builder.charge = Int(parameters['charge'])
    
    ase_geom =  structure_selector.structure        

    # spin guess
    mag_list = [ 0 for t in ase_geom ]
    if parameters['multiplicity']>0:
        for i in awb.utils.string_range_to_list(parameters['spin_u'])[0]:
            mag_list[i] = 1
        for i in awb.utils.string_range_to_list(parameters['spin_d'])[0]:
            mag_list[i] = -1 
        builder.multiplicity = Int(parameters['multiplicity'])
        
    builder.magnetization_per_site = List(list=mag_list)    
    builder.vdw = Bool(parameters['vdw_switch'])
    builder.protocol = Str(parameters['protocol'])
    builder.options = {
            "max_wallclock_seconds": resources.walltime_seconds,
            "resources": {
                'num_machines': resources.nodes,
                'num_mpiprocs_per_machine': resources.tasks_per_node,
                'num_cores_per_mpiproc': resources.threads_per_task,
            },
        }
    builder.metadata.label = 'Cp2kBulkOptWorkChain'
    builder.metadata.description = parameters['description']

    return builder

In [ ]:
def after_submission(_=None):   
    structure_selector.value = None
    
btn_submit_mol_opt = awb.SubmitButtonWidget(Cp2kMoleculeOptWorkChain, 
                                input_dictionary_function=prepare_mol_opt
                               )
btn_submit_mol_opt.on_submitted(after_submission)

btn_submit_slab_opt = awb.SubmitButtonWidget(Cp2kSlabOptWorkChain, 
                                input_dictionary_function=prepare_slab_opt
                               )
btn_submit_slab_opt.on_submitted(after_submission)

btn_submit_bulk_opt = awb.SubmitButtonWidget(Cp2kBulkOptWorkChain, 
                                input_dictionary_function=prepare_bulk_opt
                               )
btn_submit_bulk_opt.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit_mol_opt.btn_submit.disabled=True 
    btn_submit_slab_opt.btn_submit.disabled=True
    btn_submit_bulk_opt.btn_submit.disabled=True

def create(_=None):
    if computational_resources.value is None:
        msg = 'Please selecte a code first.'
        can_submit = False
    else:            
        can_submit, msg, details_dict = input_details.return_final_dictionary()
 
    with output:
        clear_output()
        if can_submit:
            print("Ready to submit a ",details_dict['workchain']," type of calculation" )
            if details_dict['workchain'] == 'Cp2kMoleculeOptWorkChain':
                btn_submit_mol_opt.btn_submit.disabled = not can_submit
                display(btn_submit_mol_opt)
            elif details_dict['workchain'] == 'Cp2kSlabOptWorkChain':
                btn_submit_slab_opt.btn_submit.disabled = not can_submit
                display(btn_submit_slab_opt)  
            elif details_dict['workchain'] == 'Cp2kBulkOptWorkChain':
                btn_submit_bulk_opt.btn_submit.disabled = not can_submit
                display(btn_submit_bulk_opt) 
            else:
                print('something went wrong')
        else:
            print(msg)

create_input.on_click(create)
input_details.observe(update_all, names='details')

In [ ]:
display(ipw.VBox([input_details,computational_resources,create_input,ipw.VBox([res,estimate_nodes_button])]), output)

In [13]:
# Resources estimation.
resources_estimation = ResourcesEstimatorWidget()
resources_estimation.link_to_resources_widget(resources)
ipw.dlink((empa_viewer, 'details'), (resources_estimation, 'details'))
ipw.dlink((input_details, 'uks'), (resources_estimation, 'uks'))
_ = ipw.dlink((computational_resources, 'value'), (resources_estimation, 'selected_code'))

'677ef55c-eba5-47ad-962e-4fddd2f08d68'

In [ ]:
display(ipw.VBox([input_details, resources, resources_estimation, computational_resources, create_input]), output)